In [2]:
import numpy as np
import cv2

In [4]:
def sigmoid(x):
    x = x.astype(np.float64)
    return 1/(1+np.exp(-x))
def relu(x):
    x = x.astype(np.float64)
    x = np.vstack((np.zeros(x.shape[0]),x))
    x = np.max(x,axis=0)
    return x
def leakyrelu(x,k=0.4):
    x = x.astype(np.float64)
    x[x < 0.0] = x[x < 0.0]*k
    # x[x > 0.0] = x[x > 0.0]
    return x
def randomrelu(x,low = 0.0,high = 1.0):
    x = x.astype(np.float64)
    x[x < 0.0] = x[x < 0.0]*np.random.uniform(low,high,None)
    #None can be replaced by any size of matrix
    return x

a = np.array([-1,2,3])
print(leakyrelu(a))

[-0.4  2.   3. ]


In [3]:
def mseLoss(y_true,y_pred):
    return ((y_true-y_pred)**2.0).mean()
y_true = np.array([1,0,0,1])
y_pred = np.array([0,0,0,0])
print(mseLoss(y_true,y_pred))

0.5


In [5]:
class Neuron:
    def __init__(self,weights=0.0,bias=0.0):
        self.weights = weights
        self.bias = bias
    def feedforward(self,inputs):
        total = np.dot(self.weights,inputs) + self.bias
        #Hardmard method and sum them together finall add bias
        return sigmoid(total)
        #the return method is a summary result and have a active function

In [7]:
def derivsigmoid(x):
    # x = x.astype(np.float64)
    # x = 1/(1+np.exp(-x))
    # x = x*(1-x)
    return x*(1-x)
# for the input already is sigmoid(x)
class NeuralNetwork:
    # 2 inputs
    # with a hidden layer
    def __init__(self):
        self.h1 = Neuron([0,0],0)
        self.h2 = Neuron([0,0],0)
        self.o1 = Neuron([0,0],0)
    
    def feedforward(self,x):
        out_h1 = self.h1.feedforward(x)
        out_h2 = self.h2.feedforward(x)
        out_o1 = self.o1.feedforward(np.array([out_h1,out_h2]))
        # print(self.o1.weights[0],self.o1.weights[1]) show the real weights in o1
        return out_o1
    
    def train(self,data,all_y_trues):
        learn_rate = 0.1
        epochs = 1000
        
        for epoch in range(epochs):
            for x,y_true in zip(data,all_y_trues):
                out_h1 = self.h1.feedforward(x)
                out_h2 = self.h2.feedforward(x)
                out_o1 = self.o1.feedforward(np.array([out_h1,out_h2]))
                
                y_pred = out_o1
                d_L_d_ypred = -2*(y_true-y_pred)
                # because of the MSE LOSS which means L(Loss) equals (y_true-y_pred)**2
                
                #Neuron o1
                d_ypred_d_o1w1 = out_h1*derivsigmoid(out_o1)
                d_ypred_d_o1w2 = out_h2*derivsigmoid(out_o1)
                d_ypred_d_b3 = derivsigmoid(out_o1)
                
                d_ypred_d_h1 = self.o1.weights[0] * derivsigmoid(out_o1)
                d_ypred_d_h2 = self.o1.weights[1] * derivsigmoid(out_o1)
                
                #Neuron h1
                d_h1_d_h1w1 = x[0] * derivsigmoid(out_h1)
                d_h1_d_h1w2 = x[1] * derivsigmoid(out_h1)
                d_h1_d_b1 = derivsigmoid(out_h1)
                
                #Neuron h2
                d_h2_d_h2w1 = x[0] * derivsigmoid(out_h2)
                d_h2_d_h2w2 = x[1] * derivsigmoid(out_h2)
                d_h2_d_b2 = derivsigmoid(out_h2)
                
                #start to maintence
                #Neuron h1
                self.h1.weights[0] -= learn_rate*d_L_d_ypred*d_ypred_d_h1*d_h1_d_h1w1
                self.h1.weights[1] -= learn_rate*d_L_d_ypred*d_ypred_d_h1*d_h1_d_h1w2
                self.h1.bias -= learn_rate*d_L_d_ypred*d_ypred_d_h1*d_h1_d_b1
                
                #Neuron h2
                self.h2.weights[0] -= learn_rate*d_L_d_ypred*d_ypred_d_h2*d_h2_d_h2w1
                self.h2.weights[1] -= learn_rate*d_L_d_ypred*d_ypred_d_h2*d_h2_d_h2w2
                self.h2.bias -= learn_rate*d_L_d_ypred*d_ypred_d_h2*d_h2_d_b2
                
                #Neuron o1
                self.o1.weights[0] -= learn_rate*d_L_d_ypred*d_ypred_d_o1w1
                self.o1.weights[1] -= learn_rate*d_L_d_ypred*d_ypred_d_o1w2
                self.o1.bias -= learn_rate*d_L_d_ypred*d_ypred_d_b3
                
                if epoch % 10 == 0:
                    y_pred = np.apply_along_axis(self.feedforward,1,data)
                    # with dimension = 1 in col way to get function feedforward used in data Dataset
                    print(y_pred)
                    loss = mseLoss(all_y_trues,y_pred)
                    print("Epoch: %d loss: %.3f" % (epoch,loss))
        



In [8]:
data = np.array([
    [-2,-1],
    [25,6],
    [17,4],
    [-15,-6],
])
all_y_trues = np.array([
    1,
    0,
    0,
    1,
])
network = NeuralNetwork()
network.train(data,all_y_trues)


[0.5093739 0.5093739 0.5093739 0.5093739]
Epoch: 0 loss: 0.250
[0.49982747 0.49982911 0.49982862 0.49982666]
Epoch: 0 loss: 0.250
[0.49050611 0.49059412 0.4905681  0.49046254]
Epoch: 0 loss: 0.250
[0.50011344 0.5001123  0.50011264 0.50011398]
Epoch: 0 loss: 0.250
[0.52007048 0.51188347 0.51399716 0.52432127]
Epoch: 10 loss: 0.246
[0.51218655 0.50400559 0.50602953 0.51649081]
Epoch: 10 loss: 0.245
[0.50400504 0.4965219  0.49831486 0.50797972]
Epoch: 10 loss: 0.246
[0.51485914 0.50390787 0.50646537 0.52071584]
Epoch: 10 loss: 0.244
[0.58761257 0.49055101 0.49417422 0.64138737]
Epoch: 20 loss: 0.196
[0.58170508 0.48440206 0.48796443 0.63575759]
Epoch: 20 loss: 0.195
[0.57584909 0.47823958 0.48160163 0.63008086]
Epoch: 20 loss: 0.194
[0.58529573 0.48252739 0.48591688 0.641766  ]
Epoch: 20 loss: 0.192
[0.64172108 0.45430348 0.45589809 0.72376754]
Epoch: 30 loss: 0.155
[0.63654541 0.44872439 0.45031074 0.71924221]
Epoch: 30 loss: 0.154
[0.63150389 0.44321085 0.44475767 0.71470593]
Epoch: 30 

In [9]:
x = np.array([66,1000])
x = network.feedforward(x)

if x > 0.5:
    print("she is a girl")
else:
    print("he is a boy")


he is a boy
